In [ ]:
%run ../setup.ipynb

In [ ]:
%run C:\Users\kouamea\CONNECTIONS\plots.ipynb

In [ ]:
data_directory = '../DATA\\DSC2\\'

state_data = pd.read_csv(data_directory+'/Non Controls States1.12.21.csv').iloc[:,1:]
state_data.head()# = state_data.groupby('State').nunique()

In [ ]:
state_data_noAl_Hw = state_data[(state_data.State != 'Alaska') &(state_data.State != 'Hawaii')]
state_data_noAl_Hw.head()

In [ ]:
# %load plotnine_map_codes

import pandas as pd
states = pd.read_csv('C:/Users\kouamea\CONNECTIONS\states.csv').rename(columns = {'region':'State'})#[['ID','X','Y']]
states = states.rename(columns = {'full':'State', 'x':'long','y':'lat'})[['State','long','lat', 'group']]
states.State = [i.lower() for i in states.State]
#terr = pd.read_csv('C:/Users\kouamea\CONNECTIONS\states	err.csv').rename(columns = {'region':'State'})#[['ID','X','Y']]

#states_and_terr = states#.rename(columns = {'region':'State'})
#states.head()

def centroid(DF):
    
    centroids_df = pd.DataFrame(columns = ['State','x','y'])
    for s in DF.State.unique():
       # print(s)
        df = DF[DF.State == s]
        x_list = [x for x in df['long']]
        y_list = [x for x in df['lat']]
        l = len(df)
        #print(l)
        x = sum(x_list) / l
        #print(x)
        y = sum(y_list) / l
        df = pd.DataFrame({'State':[s], 'x':[x],'y':[y]})
        
        centroids_df = pd.concat([centroids_df, df]).drop_duplicates()
    #display(centroids_df)
        
    return(centroids_df)

states_centroids = centroid(states[['State','long','lat']])
states_centroids = centroid(states[['State','long','lat']])
states_centroids.head()

### adding centroids for us terriories AND no state data. so i can annotate on map

us_terr_x = float(states_centroids[states_centroids['State'] == 'arizona']['x'].values)
us_terr_y = float(states_centroids[states_centroids['State'] == 'alaska']['y'].values)

states_centroids_terr = pd.DataFrame({'State': ['US Territories', 'No State Data'], 
                                      'x': [us_terr_x,us_terr_x-10000], 
                                      'y': [us_terr_y-100000,us_terr_y-150000]})
states_centroids_terr

states_centroids = pd.concat([states_centroids_terr, states_centroids])
#states_centroids.head()

def get_map_data(state_data):
    
    #############################################################
    df = state_data[['pid','State']].drop_duplicates()
    df.State = [i.lower() for i in df.State]
    df = df.groupby('State', as_index = False).nunique()
    df1 = df.merge(states)

    #######################################################
    df2 = df
    df2['pid'] = df2['pid'].fillna(0).astype('int64')
    #df2['%'] = [str(round((int(i)/int(1200)*100),1))+'%' for i in df2['pid']]
    #df2['%'] = ['('+ str(i)+')' for i in df2['%']]

    df2.loc[df2.pid <20, 'pid2'] = '<20'
    df2.loc[df2.pid >=20, 'pid2'] = [format(int(i), "7,d").strip(' ') for i in df2.loc[df2.pid>=20, 'pid']]
    #df2["pid2"]  = [format(int(i), "7,d").strip(' ') for i in df2.pid.fillna(0)]
    #df2["pis2"]  = [str(df2.loc[i,'pid'])+str(df2.loc[i,'%']) for i in df2.index]

    df2 = df2.merge(states_centroids).drop_duplicates()
    df2 = df2[~df2.State.isin(['US Territories','No State Data'])]

    return df1, df2    

dummy_data = pd.DataFrame({'pid':[545,455,788], 'State':['alabama', 'colorado', 'tennessee']})
map_df1, map_df2 = get_map_data(dummy_data)


import plotnine
from plotnine import *
def get_map(df1, df2, your_title):   
    
    #go left
    df2.loc[df2.State == 'colorado', 'x']= df2[df2.State == 'colorado']['x']-7000
    df2.loc[df2.State == 'nebraska', 'x']= df2[df2.State == 'nebraska']['x']-100000
    df2.loc[df2.State == 'north dakota', 'x']= df2[df2.State == 'north dakota']['x']-50000
    df2.loc[df2.State == 'south dakota', 'x']= df2[df2.State == 'south dakota']['x']-100000
    df2.loc[df2.State == 'kansas', 'x']= df2[df2.State == 'kansas']['x']-100000
    df2.loc[df2.State == 'arkansas', 'x']= df2[df2.State == 'arkansas']['x']-9000
    df2.loc[df2.State == 'minnesota', 'x']= df2[df2.State == 'minnesota']['x']-50000
    df2.loc[df2.State == 'missouri', 'x']= df2[df2.State == 'missouri']['x']-6000
    df2.loc[df2.State == 'michigan', 'x']= df2[df2.State == 'michigan']['x']-10000
    df2.loc[df2.State == 'new york', 'x']= df2[df2.State == 'new york']['x']-10000
    df2.loc[df2.State == 'massachusetts', 'x']= df2[df2.State == 'massachusetts']['x']-90000
    df2.loc[df2.State == 'district of columbia', 'x']= df2[df2.State == 'district of columbia']['x']-80000
    df2.loc[df2.State == 'hawaii', 'x']= df2[df2.State == 'hawaii']['x']-40000
    df2.loc[df2.State == 'maryland', 'x']= df2[df2.State == 'maryland']['x']-40000
 
    #go right
    df2.loc[df2.State == 'montana', 'x']= df2[df2.State == 'montana']['x']+150000
    df2.loc[df2.State == 'washington', 'x']= df2[df2.State == 'washington']['x']+120000
    df2.loc[df2.State == 'arizona', 'x']= df2[df2.State == 'arizona']['x']+180000
    df2.loc[df2.State == 'florida', 'x']= df2[df2.State == 'florida']['x']+140000
    df2.loc[df2.State == 'south carolina', 'x']= df2[df2.State == 'south carolina']['x']+100000
    df2.loc[df2.State == 'north carolina', 'x']= df2[df2.State == 'north carolina']['x']+100000
    df2.loc[df2.State == 'michigan', 'x']= df2[df2.State == 'michigan']['x']+110000
    df2.loc[df2.State == 'new jersey', 'x']= df2[df2.State == 'new jersey']['x']+120000
    df2.loc[df2.State == 'delaware', 'x']= df2[df2.State == 'delaware']['x']+120000
    df2.loc[df2.State == 'rhode island', 'x']= df2[df2.State == 'rhode island']['x']+50000
    df2.loc[df2.State == 'hawaii', 'x']= df2[df2.State == 'hawaii']['x']+180000
    #df2.loc[df2.State == 'massachusetts', 'x']= df2[df2.State == 'massachusetts']['x']+80000
    df2.loc[df2.State == 'new hampshire', 'x']= df2[df2.State == 'new hampshire']['x']+10000
#     df2.loc[df2.State == 'minnesota', 'x']= df2[df2.State == 'minnesota']['x']-6000

    
    ## go up
    df2.loc[df2.State == 'nevada', 'y']= df2[df2.State == 'nevada']['y']+100000
    df2.loc[df2.State == 'oklahoma', 'y']= df2[df2.State == 'oklahoma']['y']+1000
    df2.loc[df2.State == 'alabama', 'y']= df2[df2.State == 'alabama']['y']+3000
    df2.loc[df2.State == 'mississippi', 'y']= df2[df2.State == 'mississippi']['y']+10000
    df2.loc[df2.State == 'maryland', 'y']= df2[df2.State == 'maryland']['y']+40000
    df2.loc[df2.State == 'new york', 'y']= df2[df2.State == 'new york']['y']+30000
    df2.loc[df2.State == 'vermont', 'y']= df2[df2.State == 'vermont']['y']+80000
    df2.loc[df2.State == 'connecticut', 'y']= df2[df2.State == 'connecticut']['y']+30000
    df2.loc[df2.State == 'alaska', 'y']= df2[df2.State == 'alaska']['y']+200000
    df2.loc[df2.State == 'indiana', 'y']= df2[df2.State == 'indiana']['y']+100000
    #df2.loc[df2.State == 'massachusetts', 'y']= df2[df2.State == 'massachusetts']['y']+10000
    
    ## go down
    df2.loc[df2.State == 'oregon', 'y']= df2[df2.State == 'oregon']['y']-100000
    df2.loc[df2.State == 'idaho', 'y']= df2[df2.State == 'idaho']['y']-100000
    df2.loc[df2.State == 'kentucky', 'y']= df2[df2.State == 'kentucky']['y']-30000
    df2.loc[df2.State == 'michigan', 'y']= df2[df2.State == 'michigan']['y']-180000
    df2.loc[df2.State == 'delaware', 'y']= df2[df2.State == 'delaware']['y']-80000
    df2.loc[df2.State == 'virginia', 'y']= df2[df2.State == 'virginia']['y']-30000
    df2.loc[df2.State == 'district of columbia', 'y']= df2[df2.State == 'district of columbia']['y']-80000
    df2.loc[df2.State == 'rhode island', 'y']= df2[df2.State == 'rhode island']['y']-50000
    df2.loc[df2.State == 'hawaii', 'y']= df2[df2.State == 'hawaii']['y']-150000


#   ###################################################################################
    plotnine.options.figure_size = (14,10) #w,l

    #lowest = format(min(df1.pid), "7,d").strip(' ')
    #highest = format(max(df1.pid), "7,d").strip(' ')

    ###################################################################################
    plot =  (ggplot()
            + geom_polygon(df1, aes(x = 'long', y = 'lat',group = 'group',
                                    fill = 'pid'), colour = "black")

            +theme_classic()
            + theme(axis_ticks= element_blank(), axis_line= element_blank(),
                    axis_text= element_blank(),
                    text= element_text(family = 'Arial')
                   )

            + scale_fill_gradient(#low = 'aliceblue', high = 'skyblue', 
                                  low = 'white', high = 'lightgray',
                                  #name = 'test', 
                                  #breaks = [min(df1.pid), 
                                   #         max(df1.pid)]
                
                                    #breaks = [1,1]
                                  )
             +theme( 
                # legend_text = element_text(size= 12, va = 'bottom', color = 'black', 
                 #                           fontweight = 'normal'), 
                # legend_title= element_text(size=8, color = 'black', fontweight = 'normal'),
                 #legend_box= element_rect(size = 12),
                # legend_title_align="center", legend_direction= 'horizontal', legend_position= 'bottom' #'bottom'
                    legend_position = 'none')
             

            + geom_text(df2[['pid2','x','y']].drop_duplicates(),
                    aes(x = 'x', y = 'y', label = 'pid2'), size = 10, color = 'black',
                    #fontweight = 'demibold'
                       ) 
             
            #+ geom_text(df2[['x','y','%']].drop_duplicates(),
            #        aes(x = 'x', y = 'y', label = "%"), size = 12) 

             
             
            + theme(title= element_text(size = 12, color = 'black'
                                   #,fontweight='bold'
                                        ,va = 'center_baseline'
                                      # ,ha = 'right'
                                       ))  

             + labs(x = '', y = '', title = your_title)

            )
    
    df3 = df2[df2.State.isin(['US Territories','No State Data'])]
    #display(df3)
    if df3.empty == False:
        display(df3)    
        plot + geom_text(df3[['pid','x','y']].drop_duplicates(),
                      aes(x = 'x', y = 'y', label = 'pid'), size = 10, color = 'black'
                     ,fontweight = 'semibold'
                     ) 

    
    plot.save(data_directory+'Serology PAPER MAP2.tif', dpi = 800, limitsize = False)

    return plot#, df2

#plot = get_map(map_df1, map_df2)
#plot



In [ ]:
centroid

In [ ]:
#map_df1, map_df2 = get_map_data(state_data.rename(columns = {'Biobank ID':'pid'}))
#map_df2

In [ ]:
## with alaska and hawai
map_df1, map_df2 = get_map_data(state_data.rename(columns = {'Biobank ID':'pid'}))
plot = get_map(map_df1, map_df2
               ,your_title = 
               '''Figure1: The number of All of Us participants with blood specimens collected January 2 to March 18, 2020 
               \navailable for serologic testing from each state, N=24,079''')

plot

In [ ]:
## with alaska and hawai
map_df1, map_df2 = get_map_data(state_data_noAl_Hw.rename(columns = {'Biobank ID':'pid'}))
plot = get_map(map_df1, map_df2
               ,your_title = 
               '''Figure1: The number of All of Us participants with blood specimens collected January 2 to March 18, 2020 
               \navailable for serologic testing from each state, N=24,079''')
plot